In [1]:
from IPython.display import Markdown, display

from ml_boilerplate_module.config import load_config
from ml_boilerplate_module.llm.brochure import create_brochure
from ml_boilerplate_module.llm.client_factory import get_llm_client

In [2]:
load_config()

In [3]:
# url = "https://huggingface.co/"
# company_name = "Hugging Face"

url = "https://anthropic.com/"
company_name = "Anthropic"

#### Create brochure using OpenAI model

In [ ]:
result = create_brochure(url=url, provider="opeanai", model="", company_name=company_name)

In [ ]:
display(Markdown(result.split("```markdown")[1].split("```")[0].strip()))

#### Create brochure using Ollama model

In [ ]:
result2 = create_brochure(url=url, provider="ollama", model="gemma3", company_name=company_name)

In [ ]:
display(Markdown(result2.split("```markdown")[1].split("```")[0].strip()))

#### Gradio example

In [7]:
import gradio as gr

In [ ]:
def stream_message(system_message: str, user_message: str, provider: str, model: str) -> str:
    client = get_llm_client(provider=provider, model=model)
    return client.send_message(system_message=system_message, user_message=user_message)


view = gr.Interface(
    fn=stream_message,
    inputs=[
        gr.Textbox(label="System Message"),
        gr.Textbox(label="User Message"),
        gr.Dropdown(label="Provider", choices=["openai", "anthropic"]),
        gr.Dropdown(label="Model", choices=["gpt-4o-mini", "gpt-4.1", "claude-3-5-sonnet-20240620"]),
    ],
    outputs=[gr.Markdown(label="Response")],
    flagging_mode="never",
)
view.launch()

In [ ]:
client = get_llm_client(provider="anthropic", model="claude-3-5-sonnet-20240620")
send_message = client.send_message
stream_message = client.stream_message

In [ ]:
client = get_llm_client(provider="openai", model="gpt-4o-mini")
client.send_message(
    system_message="You are a helpful assistant that responds in markdown.",
    user_message="What is today's date?",
)

In [ ]:
view = gr.Interface(
    fn=stream_message,
    inputs=[gr.Textbox(label="System Message"), gr.Textbox(label="User Message")],
    outputs=[gr.Markdown(label="Response")],
    flagging_mode="never",
)
view.launch()

#### Conversational AI - aka Chatbot!

In [1]:
import gradio as gr

from ml_boilerplate_module import load_config
from ml_boilerplate_module.llm.anthropic_client import AnthropicAIClient
from ml_boilerplate_module.llm.chat import Chatbot
from ml_boilerplate_module.llm.google_client import GoogleAIClient
from ml_boilerplate_module.llm.openai_client import OpenAIClient

In [2]:
load_config()

In [ ]:
google_client = GoogleAIClient(model="gemini-2.0-flash")
chatbot = Chatbot(client=google_client)

In [3]:
openai_client = OpenAIClient()
chatbot = Chatbot(openai_client)

Using OpenAI model: gpt-4o-mini


In [ ]:
anthropic_client = AnthropicAIClient()
chatbot = Chatbot(anthropic_client)

In [4]:
system_message = """
You are a machine learning professor proficient in teaching graduate level students
in a top American university. You are good at using detailed mathematical concepts
and notations along with examples to ensure students understand machine learning deeply.
Response should be in markdown format.
"""

# user_message = """Explain regularization in logistic regression.
# Feel free to dive into as much detail as required to explain the concept
# at masters or PhD level."""

# chatbot.add_message(role="system", content=system_message)

In [ ]:
# chatbot.send_message()

In [5]:
def stream(message, history):
    print("Message: ", message)
    print("History: ", history)
    response = chatbot.stream_message(user_message=message, system_message=system_message)
    full_response = ""
    for chunk in response:
        full_response += chunk
        yield chunk

In [5]:
def message(message, history):
    print("Message from gradio: ")
    print(message)
    print("History from gradio: ")
    print(history)
    response = chatbot.send_message(user_message=message, system_message=system_message)
    return response

In [ ]:
gr.ChatInterface(fn=message, type="messages").launch()